In [1]:
import sys
import os

from config import Config, Model
import argparse
from predictors.svm_predictor import SVMPredictor
import logging
import json

In [2]:
cfg = Config.from_json(f"scripts/models/Model.SVM/meta.json")

INFO:ISO_DA:Corpora loaded succesfully! Loaded corpora:
INFO:ISO_DA:[]


In [3]:
cfg.out_folder = 'scripts/models/Model.SVM'

In [35]:
non = "train.src.da"
with open(non) as f:
    src = f.readlines()

In [30]:
from spacy.lang.en import English 
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [31]:
conv = []
for each in src:
    if "do you like" in each:
        conv.append(each[:-1])

In [32]:
len(conv)

15

In [92]:
src[132284: 132306]

['Statement\n',
 'Statement _eos Directive\n',
 'Statement _eos Directive _eos Statement Statement Statement\n',
 'Statement _eos Directive _eos Statement Statement Statement _eos Feedback Directive Statement\n',
 'Statement _eos Directive _eos Statement Statement Statement _eos Feedback Directive Statement _eos Statement Statement\n',
 'Statement _eos Directive _eos Statement Statement Statement _eos Feedback Directive Statement _eos Statement Statement _eos Statement\n',
 'Statement _eos Directive _eos Statement Statement Statement _eos Feedback Directive Statement _eos Statement Statement _eos Statement _eos Statement Statement\n',
 'Statement _eos Directive _eos Statement Statement Statement _eos Feedback Directive Statement _eos Statement Statement _eos Statement _eos Statement Statement _eos Feedback Statement\n',
 'Statement _eos Directive _eos Statement Statement Statement _eos Feedback Directive Statement _eos Statement Statement _eos Statement _eos Statement Statement _eos Fe

In [33]:
till_now = ""
first = True
for i in range(len(src)):
    if "do you like" in src[i]:
        line = src[i][:-1]
        tag = tag_line(line)
        if first:
            till_now = tag
            first = False
        else:
            till_now += " _eos " + tag
        src[i] = till_now + "\n"
        print(i, till_now)

8024 Statement
8025 Statement _eos Directive
8026 Statement _eos Directive _eos Statement
8027 Statement _eos Directive _eos Statement _eos Statement
8028 Statement _eos Directive _eos Statement _eos Statement _eos Statement Statement
8029 Statement _eos Directive _eos Statement _eos Statement _eos Statement Statement _eos Statement Statement
8030 Statement _eos Directive _eos Statement _eos Statement _eos Statement Statement _eos Statement Statement _eos Statement
8031 Statement _eos Directive _eos Statement _eos Statement _eos Statement Statement _eos Statement Statement _eos Statement _eos Statement
8032 Statement _eos Directive _eos Statement _eos Statement _eos Statement Statement _eos Statement Statement _eos Statement _eos Statement _eos Statement Statement
8033 Statement _eos Directive _eos Statement _eos Statement _eos Statement Statement _eos Statement Statement _eos Statement _eos Statement _eos Statement Statement _eos Feedback
8034 Statement _eos Directive _eos Statement _

In [34]:
with open('new_valid_rare.src.da', 'w') as filehandle:
    for each in src:
        filehandle.writelines("%s" % each)

In [12]:
def tag_line(line):
    turns = line.split("_eos")[:-1]
    tags = ""
    previous = ""
    for i in range(len(turns)-1, len(turns)):
        doc = nlp(turns[i])
        sentences = [sent.string.strip() for sent in doc.sents]
        for each in sentences:
            output = tagger.dialogue_act_tag(each, previous)
            if len(output) != 0:
                tag = output[0]['communicative_function']
            else:
                tag = 'NoDiaogAct'
            tags = tags + tag + " "
            previous = each
        if i < len(turns) - 1:
            tags = tags + "_eos "
    return tags[:-1]